# コーパスの読み込み
コーパス corpus :: 自然言語で記述あるいは口述され、コンピュータ上に格納されたデータのこと

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import requests

## ファイル読み込みの基礎
txtファイルを作成（`encoding="utf-8"` as default）、`open`で展開（オブジェクトを返す）、`read()`で読み込み（全体を読み込むため２回目の`read()`は空文字列を返す）、`close`で閉じる

In [8]:
txt = [
    """
    こんにちは。
    ファイル読み込み練習中。
    Awesome!
    """
]

In [9]:
np.savetxt("section3_example.txt", txt, fmt="%s")

In [10]:
f = open("section3_example.txt", "r", encoding="utf-8")
f

<_io.TextIOWrapper name='section3_example.txt' mode='r' encoding='utf-8'>

In [11]:
f.read()

'\n    こんにちは。\n    ファイル読み込み練習中。\n    Awesome!\n    \n'

In [12]:
f.read()

''

In [19]:
f.close()

もし、ファイルのencodingが一致していない場合、`UnicodeDecodeError`を吐く。その場合、`open`でencodingを正しく指定するか、ファイルエンコーディングを使用したいものに変更することで回避できる

In [14]:
np.savetxt("section3_example-euc.txt", txt, fmt="%s", encoding="EUC-JP")

In [17]:
f = open("section3_example-euc.txt", "r", encoding="utf-8")
f.read()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa4 in position 5: invalid start byte

In [18]:
f = open("section3_example-euc.txt", "r", encoding="EUC-JP")
f.read()

'\n    こんにちは。\n    ファイル読み込み練習中。\n    Awesome!\n    \n'

`readline()`を使用することで、一行ずつ読み込むことも可能

In [36]:
f = open("section3_example.txt", "r", encoding="utf-8")
for row_no in range(7):
    line = f.readline()
    print(f"{row_no+1}行目(文字数；{len(line)})： {line}" )
f.close()

1行目(文字数；1)： 

2行目(文字数；11)：     こんにちは。

3行目(文字数；17)：     ファイル読み込み練習中。

4行目(文字数；13)：     Awesome!

5行目(文字数；5)：     

6行目(文字数；0)： 
7行目(文字数；0)： 


オブジェクトをforで回すことも可能

In [38]:
f = open("section3_example.txt", "r", encoding="utf-8")
for row_no, line in enumerate(f):
    print(f"{row_no+1}行目(文字数；{len(line)})： {line}" )
f.close()

1行目(文字数；1)： 

2行目(文字数；11)：     こんにちは。

3行目(文字数；17)：     ファイル読み込み練習中。

4行目(文字数；13)：     Awesome!

5行目(文字数；5)：     



`with`文を使用すれば`close`する手間が省ける

In [39]:
with open("section3_example.txt", "r", encoding="utf-8") as f:
    for row_no, line in enumerate(f):
        print(f"{row_no+1}行目(文字数；{len(line)})： {line}" )

1行目(文字数；1)： 

2行目(文字数；11)：     こんにちは。

3行目(文字数；17)：     ファイル読み込み練習中。

4行目(文字数；13)：     Awesome!

5行目(文字数；5)：     



`mode="w"`にすることでファイルへの書き込みも可能（`mode="a"`で追加）

In [40]:
with open("section3_contents.txt", "w", encoding="utf-8") as f:
    f.write("Hello write method!")
!cat section3_contents.txt

Hello write method!

## CSV/TSVの読み込み
CSV Comma-Separated Values  
TSV Tab-Separated Values  

### csvの取り扱い

In [49]:
pd.DataFrame({"name":["高橋", "佐藤", "田中"], "grade":["B", "C", "A"]}
            ).to_csv("section3_example.csv", sep=",", encoding="utf-8", index=False)

In [50]:
pd.read_csv("section3_example.csv", encoding="utf-8")

,name,grade
0,高橋,B
1,佐藤,C
2,田中,A


headerがない場合には、一行目のデータがheaderと見做されてしまうため、headerがない旨を明記する必要

In [54]:
no_header_csv = [
    ["高橋", "B"],
    ["佐藤", "C"],
    ["田中","A"]
]
np.savetxt("section3_example1.csv", no_header_csv, fmt="%s", delimiter=",")

In [55]:
pd.read_csv("section3_example1.csv", encoding="utf-8")

,高橋,B
0,佐藤,C
1,田中,A


In [56]:
pd.read_csv("section3_example1.csv", encoding="utf-8", header=None)

,0,1
0,高橋,B
1,佐藤,C
2,田中,A


カラム名が付いていないと扱いにくいので、何かしらの手法で追加してやったほうが都合がいい（個人的には２をよく使用している;2だとどこを何に変えたかが直感的にわかりやすくエラーを減らせると思っている）

In [57]:
# 1: use names arg
display(pd.read_csv("section3_example1.csv", encoding="utf-8", header=None, names=("name", "grade")))

# 2: use rename method
display(pd.read_csv("section3_example1.csv", encoding="utf-8", header=None).rename(columns={0:"name", 1:"grade"}))

,name,grade
0,高橋,B
1,佐藤,C
2,田中,A


,name,grade
0,高橋,B
1,佐藤,C
2,田中,A


### tsvの取り扱い
`read_csv`で`sep="\t"`にするか、`read_table`を使用することで対応可能

In [58]:
pd.DataFrame({"name":["高橋", "佐藤", "田中"], "grade":["B", "C", "A"]}
            ).to_csv("section3_example.tsv", sep="\t", encoding="utf-8", index=False)

In [60]:
pd.read_csv("section3_example.tsv", encoding="utf-8")

,name\tgrade
0,高橋\tB
1,佐藤\tC
2,田中\tA


In [61]:
pd.read_csv("section3_example.tsv", encoding="utf-8", sep="\t")

,name,grade
0,高橋,B
1,佐藤,C
2,田中,A


In [62]:
pd.read_table("section3_example.tsv", encoding="utf-8")

,name,grade
0,高橋,B
1,佐藤,C
2,田中,A


## JSONの読み込み

In [68]:
json_exm = """[
    {
        "name": "高橋",
        "grade": "B"
    },
    {
        "name": "佐藤",
        "grade": "C"
    },
    {
        "name": "田中",
        "grade": "A"
    }
]"""
with open("section3_example.json", "w", encoding="utf-8") as f:
    f.write(json_exm)

In [69]:
pd.read_json("section3_example.json", encoding="utf-8")

,name,grade
0,高橋,B
1,佐藤,C
2,田中,A


In [71]:
json_exml = """
    {"name": "高橋","grade": "B"}
    {"name": "佐藤","grade": "C"}
    {"name": "田中","grade": "A"}
"""
with open("section3_example.jsonl", "w", encoding="utf-8") as f:
    f.write(json_exml)

In [78]:
try:
    pd.read_json("section3_example.jsonl", encoding="utf-8")
except ValueError as ve:
    print("got ValueError: ", ve)
    display(pd.read_json("section3_example.jsonl", encoding="utf-8", lines=True))

got ValueError:  Trailing data


,name,grade
0,高橋,B
1,佐藤,C
2,田中,A


## ディレクトリの走査

In [80]:
!mkdir sec3_dir1
!touch sec3_dir1/example1.txt sec3_dir1/example2.txt sec3_dir1/example3.txt
!mkdir sec3_dir1/sec3_dir2
!touch sec3_dir1/sec3_dir2/example4.txt sec3_dir1/sec3_dir2/example5.txt

mkdir: cannot create directory ‘sec3_dir1’: File exists


In [81]:
p = Path("sec3_dir1")
p

PosixPath('sec3_dir1')

In [82]:
list(p.glob('*'))

[PosixPath('sec3_dir1/example1.txt'),
 PosixPath('sec3_dir1/example2.txt'),
 PosixPath('sec3_dir1/example3.txt'),
 PosixPath('sec3_dir1/sec3_dir2')]

In [83]:
list(p.glob('*.txt'))

[PosixPath('sec3_dir1/example1.txt'),
 PosixPath('sec3_dir1/example2.txt'),
 PosixPath('sec3_dir1/example3.txt')]

In [84]:
list(p.glob('**/*.txt'))

[PosixPath('sec3_dir1/example1.txt'),
 PosixPath('sec3_dir1/example2.txt'),
 PosixPath('sec3_dir1/example3.txt'),
 PosixPath('sec3_dir1/sec3_dir2/example4.txt'),
 PosixPath('sec3_dir1/sec3_dir2/example5.txt')]

# コーパスの作成

In [13]:
key_dict = pd.read_json("../gnavi_keys.json", encoding="utf-8")
key_dict

,api_key,access_key
0,0c41b644a28dd78fae56c772936feece,0c41b644a28dd78fae56c772936feece


In [14]:
url = "https://api.gnavi.co.jp/PhotoSearchAPI/v3/"
params = {"keyid": key_dict["api_key"], "menu_name":"ラーメン"}
response = requests.get(url, params=params)
response.json()

{'response': {'@attributes': {'api_version': 'v3'},
  'total_hit_count': 5006,
  'hit_per_page': 15,
  '0': {'photo': {'vote_id': '79812',
    'photo_genre_id': '1',
    'photo_genre_name': '料理・ドリンク',
    'photo_scene_id': '',
    'photo_scene_name': '',
    'nickname': 'ＦＲＢ',
    'shop_id': '7377127',
    'shop_name': '一蘭町田店',
    'shop_url': 'https://r.gnavi.co.jp/34n4jgkb0000/?ak=iijEWPZ0uuZKZQQpvJPoJ3ZdDa50nhWZBtGB55KwqaU%3D',
    'prefname': 'PREF13:東京都',
    'menu_id': 78256,
    'menu_name': 'ラーメン',
    'menu_finish_flag': 0,
    'areaname_l': '町田・多摩',
    'areaname_m': '町田',
    'areaname_s': '町田',
    'image_url': {'url_1024': 'https://mr.gnavi.co.jp/cont/menu_image/e6/41/79812_l.jpeg',
     'url_320': 'https://mr.gnavi.co.jp/cont/menu_image/e6/41/79812.jpeg',
     'url_250': 'https://mr.gnavi.co.jp/cont/menu_image/e6/41/79812_m.jpeg',
     'url_200': 'https://mr.gnavi.co.jp/cont/menu_image/e6/41/79812_s.jpeg'},
    'comment': '超こってりラーメンががうまい。なんか特許があるらしい。',
    'total_score': 

`corpus_maker.py`をscriptとして作成する  
`src/scripts/chapter03/data/raw_data.json`, `src/scripts/chapter03/data/dataset.jsonl`を作成

In [4]:
rawdata_path = "../scripts/chapter03/data/raw_data.json"
dataset_path = "../scripts/chapter03/data/dataset.jsonl"

クローリングした結果

In [6]:
pd.read_json(rawdata_path, encoding="utf-8")

,response
0,"{'photo': {'vote_id': '77995', 'photo_genre_id..."
1,"{'photo': {'vote_id': '77804', 'photo_genre_id..."
10,"{'photo': {'vote_id': '87602', 'photo_genre_id..."
11,"{'photo': {'vote_id': '88712', 'photo_genre_id..."
12,"{'photo': {'vote_id': '89852', 'photo_genre_id..."
13,"{'photo': {'vote_id': '89853', 'photo_genre_id..."
14,"{'photo': {'vote_id': '90839', 'photo_genre_id..."
15,"{'photo': {'vote_id': '91009', 'photo_genre_id..."
16,"{'photo': {'vote_id': '91012', 'photo_genre_id..."
17,"{'photo': {'vote_id': '91600', 'photo_genre_id..."


スクレイピングした結果

In [8]:
pd.read_json(dataset_path, encoding="utf-8", lines=True)

,comment,score
0,シチューで煮込んだロールキャベツ。大きい肉がたっぷりのキャベツにくるまれて、シチューも洋食屋...,3
1,"ドライカレーのランチ！オードブル、コーヒー、冷製スープがついて、セットで1,200円です。\...",3
2,濃厚です！ボリューミーです！！美味です！！！,3
3,極上！！めちゃウマなのに安い！！店員さんもよく気がついてくれます。新宿三丁目店お気に入りです。,3
4,もつ鍋というものをはじめて食べたお店です。後味がさっぱりしていて、くどくなくて美味しかったです。,3
5,レバ刺しをはじめて食べたお店です☆ジューシーでくさみがなくておいしい！,3
6,ふかひれがぎっしり,3
7,野菜がしゃきしゃき,3
8,以前はグランドメニューに無かった裏メニューだと勝手に思っていたが、先日来店したらグランドメニ...,3
9,東京で食べられるイカでは最高峰だと思います。新鮮。量もがっつりあります。おすすめ。,3
